## Load data for training word embeddings
<hr>

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# read data

data = pd.read_csv('data/processed_data_model1.csv')
print(data.shape)
data.head()

(58887, 7)


,id,title,corpus,score,polarity,subjectivity,processed_title
0,21454,Specifying a mySQL ENUM in a Django model,specifying mysql enum django model go specifyi...,0.044573,0.0,0.00,specifying mysql enum django model
1,22059,"How do content discovery engines, like Zemanta...",content discovery engines like zemanta open ca...,0.002254,0.0,0.00,content discovery engines like zemanta open ca...
2,29856,Install Python to match directory layout in OS...,install python match directory layout os x 105...,-0.000657,0.0,0.00,install python match directory layout os x 105
3,35569,Why does Python's iter() on a mapping return i...,python iter mapping return iterkeys instead it...,0.001134,-0.2,0.85,python iter mapping return iterkeys instead it...
4,39960,Javascript equivalent of Python's locals()?,javascript equivalent python locals python one...,0.003821,0.0,0.00,javascript equivalent python locals


## Training word embeddings
<hr>

In [3]:
import multiprocessing
from gensim.models import Word2Vec

In [4]:
cores = multiprocessing.cpu_count()
print(cores)

8


In [5]:
import gensim

w2v_model = Word2Vec(min_count=2,
                                         window=2,
                                         size=300,
                                         sample=0, 
                                         alpha=0.03, 
                                         min_alpha=0.0007, 
                                         negative=10,
                                         workers=cores-1)

corpus = [text.split() for text in np.array(data.corpus)]

In [6]:
#train word embeddings

w2v_model.build_vocab(corpus) #build the vocabulary
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("vocab size",vocab_size)
w2v_model.train(corpus, total_examples=len(corpus), epochs=32) #training step

vocab size 74743


(81551392, 87125344)

In [7]:
w2v_model.save('models/related_questions_model.bin') #saving the trained w2v model

## Sanity check on the trained word embeddings
<hr>

In [8]:
print('Terms most similar to "flask" \n')
print(w2v_model.wv.most_similar("flask"))

Terms most similar to "flask" 

[('django', 0.5730941891670227), ('pyramid', 0.4540061950683594), ('server', 0.4248735010623932), ('gae', 0.4246968626976013), ('app', 0.4185467064380646), ('webpy', 0.41662949323654175), ('cherrypy', 0.41528671979904175), ('web', 0.3963538408279419), ('fastapi', 0.3919406533241272), ('login', 0.3860347867012024)]


In [9]:
print('Terms most similar to "django" \n')
print(w2v_model.wv.most_similar("django"))

Terms most similar to "django" 

[('flask', 0.5730941891670227), ('python', 0.488075315952301), ('template', 0.4599888324737549), ('pyramid', 0.4333301782608032), ('web2py', 0.42985913157463074), ('drf', 0.4194490313529968), ('sqlalchemy', 0.41887274384498596), ('app', 0.40341660380363464), ('database', 0.3992598354816437), ('wagtail', 0.3980596661567688)]


In [10]:
print('Terms most similar to "api" \n')
print(w2v_model.wv.most_similar("api"))

Terms most similar to "api" 

[('apis', 0.4807102680206299), ('interface', 0.4239417314529419), ('service', 0.4118573069572449), ('function', 0.39397263526916504), ('script', 0.39292168617248535), ('code', 0.3902546167373657), ('oauth', 0.388918399810791), ('restful', 0.3885675370693207), ('appengine', 0.38507091999053955), ('app', 0.38147079944610596)]


## Relevant questions retrieval model
<hr>

In [11]:
import os
import numpy as np
import pandas as pd

## Load data
<hr>

In [5]:
data = pd.read_csv('data/processed_data_model1.csv')
data

,id,title,corpus,score,polarity,subjectivity,processed_title
0,21454,Specifying a mySQL ENUM in a Django model,specifying mysql enum django model go specifyi...,0.044573,0.000000,0.000000,specifying mysql enum django model
1,22059,"How do content discovery engines, like Zemanta...",content discovery engines like zemanta open ca...,0.002254,0.000000,0.000000,content discovery engines like zemanta open ca...
2,29856,Install Python to match directory layout in OS...,install python match directory layout os x 105...,-0.000657,0.000000,0.000000,install python match directory layout os x 105
3,35569,Why does Python's iter() on a mapping return i...,python iter mapping return iterkeys instead it...,0.001134,-0.200000,0.850000,python iter mapping return iterkeys instead it...
4,39960,Javascript equivalent of Python's locals()?,javascript equivalent python locals python one...,0.003821,0.000000,0.000000,javascript equivalent python locals
...,...,...,...,...,...,...,...
58882,65163096,Comparing values with __eq__ in Python,comparing values _ _ eq _ _ python good day ni...,-0.001329,-0.083333,0.283333,comparing values _ _ eq _ _ python
58883,65163335,How to get PYQT5 text box to be taken as an in...,get pyqt5 text box taken input sqlite trying c...,-0.001105,0.000000,0.000000,get pyqt5 text box taken input sqlite
58884,65163681,How to assign one argument to function and mak...,assign one argument function make list later q...,-0.001329,0.000000,0.000000,assign one argument function make list later
58885,65163947,Iterate over a list based on list based on a l...,iterate list based list based list steps want ...,0.000686,-0.055556,0.307937,iterate list based list based list steps


In [6]:
import re
import nltk
#import inflect
from nltk.corpus import stopwords

#process text

import spacy
en = spacy.load('en_core_web_sm')

import nltk
nltk.download('stopwords')

#tokenizer

def tokenize(text):
    tokens = en.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

#remove punctuations

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words 

#remove stop words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def process_text(text):
    return ' '.join(normalize(tokenize(text)))

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

## Import saved word embeddings
<hr>

In [7]:
#getting the saved word embeddings

from gensim.models import Word2Vec

w2v_model = Word2Vec.load('models/related_questions_model.bin')
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)

print("vocab size",vocab_size)

## Calculate sentence embeddings
<hr>

In [8]:
#word to numerical vector using the trained word embeddings

def question_to_vec(question, embeddings, dim = 300):
    question_embedding = np.zeros(dim) #initialize with zeros and dim = 300
    valid_words = 0
    for word in str(question).split(' '):
        if embeddings.wv.__contains__(word):
            valid_words+=1
            question_embedding += embeddings.wv.__getitem__(word)
        if valid_words>0:
            return question_embedding/valid_words
        else:
            return question_embedding

In [ ]:
#converting all the question titles to vectors and saving them

title_embeddings = []

for title in data.processed_title:
    title_embeddings.append(question_to_vec(question = title, embeddings = w2v_model))
    
title_embeddings = np.array(title_embeddings)
embeddings = pd.DataFrame(data = title_embeddings)
embeddings[0:15000].to_csv('models/title_embeddings1.csv', index=False)
embeddings[15000:30000].to_csv('models/title_embeddings2.csv', index=False)
embeddings[30000:45000].to_csv('models/title_embeddings3.csv', index=False)
embeddings[45000:].to_csv('models/title_embeddings4.csv', index=False)

print(w2v_model)

## Similarity Score Calculation and Ranking the Questions
<hr>

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import HTML

# retrieving the save title embeddings

em1 = pd.read_csv('models/title_embeddings1.csv')
em2 = pd.read_csv('models/title_embeddings2.csv')
em3 = pd.read_csv('models/title_embeddings3.csv')
em4 = pd.read_csv('models/title_embeddings4.csv')

em = em1.append([em2, em3, em4])

title_embeddings = np.array(em)

# process the query

query = 'merge two lists in python'
processed_query = process_text(query)

results_returned = 100 # number of results to be returned

query_vect = np.array([question_to_vec(processed_query, w2v_model)]) # Vectorize the user query

cosine_similarities = pd.Series(cosine_similarity(query_vect, title_embeddings)[0]) # calculate the cosine similarities

relevant_questions = []
max_cosine_score = max(cosine_similarities)

cos_weight = 30
freq_weight = 60
ans_weight = 6
polar_weight = 2
subj_weight = 2

#score calculation

for index, cosine_score in cosine_similarities.nlargest(results_returned).iteritems():
    
    freq_score = 0
    word_count = 0
    score = 0
    
    for word in data.processed_title[index].split():
        if word.lower() in processed_query:
            freq_score+=1
        word_count+=1
        
    freq_score/=word_count
    
    score =  (cos_weight*(cosine_score/max_cosine_score)+freq_weight*freq_score+ans_weight*data.score[index]+polar_weight*data.polarity[index]+subj_weight*data.subjectivity[index])/100
        
    relevant_questions.append((index, data.id[index], data.title[index], score))
    
relevant_questions.sort(key = lambda x : x[3], reverse = True)

# display results

output = ''

for index, qid, question, score in relevant_questions:
    output += '<p style="font-family:verdana; font-size:110%;"> '
    output += 'question id : '+str(qid)+'<br>similarity score : ' + str(score) + '<br>'
    for word in question.split():
        if word.lower() in processed_query:
            output += " <b>"+str(word)+"</b>"
        else:
            output += " "+str(word)
    output += "</p><hr>"

output = '<h3>Results for query - '+query+'</h3>'+output
display(HTML(output))

# save the retrieved questions

df = pd.DataFrame(relevant_questions).iloc[:,1:]
df.columns = ['id','questions','score']

df.to_csv('data/relevant_questions.csv', index=False)

In [18]:
qdf = pd.read_csv('data/relevant_questions.csv')
qdf.head(10)

,id,questions,score
0,48641350,How to merge two lists into a list of multiple...,0.799961
1,46362972,Merge list of lists in python 3,0.785947
2,1158128,Merge sorted lists in python,0.750189
3,44476206,how to merge two list having dict,0.749920
4,58971955,How to merge two dataframe?,0.709934
5,35526224,How to merge these two Jsons?,0.701170
6,49734759,Merge two list into a list of dictionaries wit...,0.674987
7,34761978,Merge 3 lists into 1 list,0.660068
8,30620323,Merge two files in Python PANDAS?,0.660041
9,55397854,How to merge two IO Streams in Python?,0.659947


In [19]:
qdf.shape

(100, 3)